# Hybrid RAG Evaluation — Dense + Keyword (Ensemble) Retrieval

This notebook explores a **Hybrid RAG retrieval strategy** by combining:
- Dense vector similarity search (FAISS)
- Sparse keyword-based retrieval (BM25)

An ensemble retriever merges the results of both techniques, aiming to capture semantically relevant content (dense) while retaining precise keyword matches (sparse). 
This setup helps mitigate weaknesses of each method individually and improves answer coverage in certain question types.

Only the retrieval component is changed compared to the baseline. All other pipeline elements (chunking, embeddings, LLM, evaluation) are kept consistent.

In [1]:
import os

# Set environment variables
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGSMITH_PROJECT"] = "Basic_RAG_Semantic_Chunking"


# Verify that the environment variables are set
print("Environment Variables Set:")
print(f'LANGSMITH_TRACING: {os.getenv("LANGSMITH_TRACING")}')
print(f'LANGSMITH_ENDPOINT: {os.getenv("LANGSMITH_ENDPOINT")}')
print(f'LANGSMITH_PROJECT: {os.getenv("LANGSMITH_PROJECT")}')

In [2]:
import sys
import os
import re
import pandas as pd
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
import langchain
from langchain_community.document_loaders import PyPDFLoader
from langsmith import traceable
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma
from langsmith import traceable
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings
from enum import Enum
from typing import Any, Dict, List, Tuple

/Users/user/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ········


## LLM and Embedding Model

In [4]:
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    max_tokens=4000,
)

In [11]:
# Load the index back

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
new_vector_store = FAISS.load_local(
    "nvidia_large_baseline", 
    embeddings,  # Use the same embeddings used during creation
    allow_dangerous_deserialization=True  # Important for loading
)


retriever = new_vector_store.as_retriever(search_kwargs={"k": 3})

## Keyword Search Retriever 

In [6]:
def load_pdf(file_path: str):
    loader = PyPDFLoader(file_path)
    return loader.load()

# Load the PDF with tracing
# path = 'The One Page Linux Manual.pdf'

# path = 'paul_graham.pdf'

path ='NVIDIA_Report_2024.pdf'
document = load_pdf(path)

In [7]:
def split_text(document):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,
        chunk_overlap=150,
        length_function=len,
    )
    docs = text_splitter.split_documents(document)

    # Create and print a table of text chunks
    df = pd.DataFrame({
        "Chunk Index": range(len(docs)),
        "Text Chunk": [doc.page_content for doc in docs]
    })
    
    print("\n===== Top 3 Text Chunks =====\n")
    print(df.head(3).to_string(index=False))  # Print only the first 3 rows
    print("\n=============================\n")

    
    return docs


docs = split_text(document)


===== Top 3 Text Chunks =====

 Chunk Index                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     Text Chunk
           0 Table of Contents\nUNITED STATESSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\n___________________________________________________________________

In [9]:
# create keyword retriever
from langchain.retrievers import BM25Retriever
keyword_retriever = BM25Retriever.from_documents(docs)
keyword_retriever.k =  3

In [10]:
keyword_retriever.invoke("Who is the CEO of NVIDIA?")

[Document(metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2024-02-21T17:37:32-05:00', 'title': '0001045810-24-000029', 'author': 'EDGAR® Online LLC, a subsidiary of OTC Markets Group', 'subject': 'Form 10-K filed on 2024-02-21 for the period ending 2024-01-28', 'keywords': '0001045810-24-000029; ; 10-K', 'moddate': '2024-02-21T17:37:44-05:00', 'source': 'NVIDIA_Report_2024.pdf', 'total_pages': 96, 'page': 94, 'page_label': '95'}, page_content='Accounting\nRestatement Date\nThe earlier to occur of (i) the date that the Board, a committee of the Board, or the officer or officers of the Company\nauthorized to take such action if Board action is not required, concludes, or reasonably should have concluded, that the\nCompany is required to prepare an Accounting Restatement, or (ii) the date that a court, regulator or other legally\nauthorized body directs the Company to prepare an Accounting Restatement\nBoard Board of 

## Ensemble Retriever

In [12]:
# create ensemble retriever
from langchain.retrievers import EnsembleRetriever
ensemble_retriever = EnsembleRetriever(retrievers=[retriever, keyword_retriever], weights=[0.5, 0.5])

In [13]:
# test ensemble retriever
ensemble_retriever.invoke("Who is the CEO of NVIDIA?")

[Document(id='ceca20ec-4a59-4001-84ec-cc83ff8dd65c', metadata={'producer': 'EDGRpdf Service w/ EO.Pdf 22.0.40.0', 'creator': 'EDGAR Filing HTML Converter', 'creationdate': '2024-02-21T17:37:32-05:00', 'title': '0001045810-24-000029', 'author': 'EDGAR® Online LLC, a subsidiary of OTC Markets Group', 'subject': 'Form 10-K filed on 2024-02-21 for the period ending 2024-01-28', 'keywords': '0001045810-24-000029; ; 10-K', 'moddate': '2024-02-21T17:37:44-05:00', 'source': 'NVIDIA_Report_2024.pdf', 'total_pages': 96, 'page': 11, 'page_label': '12'}, page_content="Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Directo

## RAG Chain

In [14]:
def setup_rag_pipeline():
    template = """"
    You are a helpful assistant that answers questions based on the provided context.
    Use the provided context to answer the question.
    Question: {input}
    Context: {context}
    Answer:
    """
    prompt = ChatPromptTemplate.from_template(template)
    
    # Setup RAG pipeline
    return (
        {"context": ensemble_retriever, "input": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

# Initialize the RAG chain with tracing
rag_chain = setup_rag_pipeline()

In [15]:
@traceable
def query_rag_top_6(question: str, top_k=6):
    """
    Query the RAG pipeline, retrieve relevant documents, and display their similarity scores in descending order.
    Returns both the generated answer and retrieved contexts.
    """
    # Retrieve the top-k chunks with similarity scores
    retrieved_docs_with_scores = ensemble_retriever.get_relevant_documents(question, k=top_k)

    # Sort documents by similarity score in descending order
#     retrieved_docs_with_scores = sorted(retrieved_docs_with_scores, key=lambda x: x[1], reverse=True)

    # Extract document details into a structured DataFrame
    df = pd.DataFrame([
        {
            "Document Name": doc.metadata.get("source", "Unknown Document"),
            "Page Number": doc.metadata.get("page", "N/A"),
#             "Similarity Score": round(score, 4),  # Rounding for readability
            "Retrieved Chunk": doc.page_content[:1000] + "..."  # Show up to 1000 characters for clarity
        }
        for doc in retrieved_docs_with_scores
    ])

    # Ensure full content is displayed in the DataFrame
    pd.set_option("display.max_colwidth", None)  # Do not truncate column content
    pd.set_option("display.width", 1000)  # Set a larger display width for better visibility

    # Display the DataFrame in Jupyter
    display(df)

    # Extract contexts (raw document texts)
    retrieved_contexts = [doc.page_content for doc in retrieved_docs_with_scores]

    # Run the RAG pipeline with the retrieved context
    answer = rag_chain.invoke(question)

    # Return both the answer and the retrieved contexts
    return {
        "answer": answer,
        "contexts": retrieved_contexts
    }

In [16]:
# Example usage:
response = query_rag_top_6("When did the CEO enter office?")

/var/folders/8_/66zkglc51d973l0mk64lcrkc0000gn/T/ipykernel_3458/1913972663.py:8: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs_with_scores = ensemble_retriever.get_relevant_documents(question, k=top_k)


,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,84,"Robert Burgess\n/s/ TENCH COXE Director February 21, 2024\nTench Coxe \n/s/ JOHN O. DABIRI Director February 21, 2024\nJohn O. Dabiri \n/s/ PERSIS DRELL Director February 21, 2024\nPersis Drell\n/s/ DAWN HUDSON Director February 21, 2024\nDawn Hudson\n/s/ HARVEY C. JONES Director February 21, 2024\nHarvey C. Jones \n/s/ MELISSA B. LORA Director February 21, 2024\nMelissa B. Lora\n/s/ MICHAEL MCCAFFERY Director February 21, 2024\nMichael McCaffery\n/s/ STEPHEN C. NEAL Director February 21, 2024\nStephen C. Neal\n/s/ MARK L. PERRY Director February 21, 2024\nMark L. Perry \n/s/ A. BROOKE SEAWELL Director February 21, 2024\nA. Brooke Seawell \n/s/ AARTI SHAH Director February 21, 2024\nAarti Shah\n/s/ MARK STEVENS Director February 21, 2024\nMark Stevens \n85..."
1,NVIDIA_Report_2024.pdf,21,"networking products has not experienced any significant impact. Further, in connection with the conflict, a substantial number of our employees in the regionhave been called-up for active military duty in Israel. Accordingly, some of our employees in Israel have been absent for an extended period and they or others\nmay continue to be absent, which may cause disruption to our product development or operations. We did not experience any significant impact or expense toour business; however, if the conflict is further extended, it could impact future product development, operations, and revenue or create other uncertainty for ourbusiness...."
2,NVIDIA_Report_2024.pdf,11,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
3,NVIDIA_Report_2024.pdf,34,"networking products has not experienced any significant impact. Further, in connection with the conflict, a substantial number of our employees in the regionhave been called-up for active military duty in Israel. Accordingly, some of our employees in Israel have been absent for an extended period and they or othersmay continue to be absent, which may cause disruption to our product development or operations. We did not experience any significant impact or expense to\nour business; however, if the conflict is further extended, it could impact future product development, operations, and revenue or create other uncertainty for ourbusiness.\n35..."
4,NVIDIA_Report_2024.pdf,11,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."
5,NVIDIA_Report_2024.pdf,72,"dismiss the complaint without leave to amend, entered judgment in favor of NVIDIA and closed the case. On March 30, 2021, plaintiffs filed an appeal fromjudgment in the United States Court of Appeals for 

In [17]:
response["answer"]

"The CEO, Jen-Hsun Huang, co-founded NVIDIA in 1993 and has served as the President and Chief Executive Officer since the company's inception. Therefore, he entered office in 1993."

## RAG Evaluation Dataset

In [19]:
# Import the LangSmith client
from langsmith import Client

# Initialize the LangSmith client
ls_client = Client()

# Read an existing dataset by name
dataset_name = "RAG_Eval_QA"
dataset = ls_client.read_dataset(dataset_name=dataset_name)

# List examples in the dataset
examples = ls_client.list_examples(dataset_name=dataset_name)

# Iterate through examples
for example in examples:
    print(example.inputs)  # Print input data
    print(example.outputs)  # Print output data


{'question': "What was NVIDIA's revenue in 2024 and 2023? Which year saw higher revenues and by how much?"}
{'answer': "Revenue in 2024 was $60,922 million ($60.9 billion) , revenue in 2023 was $26,974 million ($27.0 billion). NVIDIA's revenue increased by $33,948 million (or $33.95 billion depending on the rounding) in 2024 compared to 2023."}
{'question': 'Between January 29, 2023, and January 28, 2024, NVIDIA’s total assets increased by approximately $24.5 billion. Based on the balance sheet, which two line items contributed the most to this increase, and what does this suggest about the company’s capital deployment focus?'}
{'answer': 'Between January 29, 2023, and January 28, 2024, NVIDIA’s total assets increased by approximately $24.5 billion, rising from $41.2 billion to $65.7 billion. Based on the balance sheet, the two line items that contributed the most to this increase were marketable securities, which increased from $9.9 billion to $18.7 billion, and cash and cash equivale

In [22]:
from langchain_core.rate_limiters import InMemoryRateLimiter

rate_limiter = InMemoryRateLimiter(
    requests_per_second=0.1,  # Slow down requests
    check_every_n_seconds=0.1,
    max_bucket_size=10
)

# Apply to your model

model = ChatOpenAI(model="gpt-4o", temperature=0, rate_limiter=rate_limiter)

/var/folders/8_/66zkglc51d973l0mk64lcrkc0000gn/T/ipykernel_3458/2168577922.py:3: LangChainBetaWarning: Introduced in 0.2.24. API subject to change.
  rate_limiter = InMemoryRateLimiter(


## RAG Evaluators

### Type 1: Reference Answer

First, lets consider the case in which we want to compare our RAG chain answer to a reference answer.

#### Eval flow

We simply use an LLM-as-judge with an easily customized grader prompt: 

In [23]:
# RAG chain
def predict_rag_answer(example: dict):
    """Use this for answer evaluation"""
    response =  query_rag_top_6(example["question"])
    return {"answer": response["answer"]}

In [24]:
def predict_rag_answer_with_context(example: dict):
    """Use this for evaluation of retrieved documents and hallucinations"""
    response = query_rag_top_6(example["question"])
    return {"answer": response["answer"], "contexts": response["contexts"]}

In [25]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
grade_prompt_answer_accuracy = hub.pull("answer_vs_reference")

In [26]:
def answer_evaluator(run, example) -> dict:
    """
    A simple evaluator for RAG answer accuracy
    """
    
    # Get summary
    input_question = example.inputs["question"]
    reference = example.outputs["answer"]
    prediction = run.outputs["answer"]

    # LLM grader
    # llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    llm = model
 
    # Structured prompt
    
    answer_grader = grade_prompt_answer_accuracy | llm

    # Get score
    score = answer_grader.invoke({"question": input_question,
                                  "correct_answer": reference,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_score", "score": score}

In [67]:
from langsmith.evaluation import evaluate

dataset_name = "RAG_test_NVIDIA"
experiment_results = evaluate(
    predict_rag_answer,
    data=dataset_name,
    evaluators=[answer_evaluator],
    experiment_prefix="Hybrid_Ensemble",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'Hybrid_Ensemble-894896c5' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/7fad5e82-7b2d-438e-90e6-1e232f388896/compare?selectedSessions=b196c74a-0c41-441b-bc7f-e4d46047b3b8




0it [00:00, ?it/s]

,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,4,"infotainment systems; and Omniverse Enterprise software for building and operating metaverse and 3D internet applications.\nOur Markets\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,interconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive.\nData Center\nThe NVIDIA Data Center platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific..."
1,NVIDIA_Report_2024.pdf,3,"serve. While the computing requirements of these end markets are diverse, we address them with a unified underlying architecture leveraging our GPUs andnetworking and software stacks. The programmable nature of our architecture allows us to support several multi-billion-dollar end markets with the sameunderlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. The large and growing number\nof developers and installed base across our platforms strengthens our ecosystem and increases the value of our platform to our customers.\nInnovation is at our core. We have invested over $45.3 billion in research and development since our inception, yielding inventions that are essential to modern..."
2,NVIDIA_Report_2024.pdf,35,"Table of Contents\nFiscal Year 2024 Summary\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Change\n($ in millions, except per share data)\nRevenue $ 60,922 $ 26,974 Up 126%\nGross margin 72.7 % 56.9 % Up 15.8 pts\nOperating expenses $ 11,329 $ 11,132 Up 2%\nOperating income $ 32,972 $ 4,224 Up 681%\nNet income $ 29,760 $ 4,368 Up 581%\nNet income per diluted share $ 11.93 $ 1.74 Up 586%\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,\ninterconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive...."


,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,35,"Revenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago.\nData Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including automotive, financial services, and healthcare. Customers across industry verticals access NVIDIA AI infrastructure both through the\ncloud and on-premises. Data Center compute revenue was up 244% in the fiscal year. Networking revenue was up 133% in the fiscal year.\nGaming revenue for fiscal year 2024 was up 15%. The increase reflects higher sell-in to partners following the normalization of channel inventory levels and\ngrowing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%...."
1,NVIDIA_Report_2024.pdf,35,"other NVIDIA RTX technologies.\nProfessional Visualization revenue for fiscal year 2024 was $1.6 billion, up 1% from fiscal year 2023. In Professional Visualization, we announced new GPUsbased on the NVIDIA RTX Ada Lovelace architecture, and announced NVIDIA Omniverse Cloud, a fully managed service running in Microsoft Azure, for the\ndevelopment and deployment of industrial metaverse applications.\nAutomotive revenue for fiscal year 2024 was $1.1 billion, up 21% from fiscal year 2023. In Automotive, we announced a partnership with MediaTek, which will\ndevelop mainstream automotive systems on chips for global OEMs integrating a new NVIDIA GPU chiplet IP for AI and graphics. We furthered our collaborationwith Foxconn to develop next-generation\n36..."
2,NVIDIA_Report_2024.pdf,38,"Graphics 5,846 4,552 1,294 28 %\nAll Other (4,890) (5,411) 521 (10)%\nTotal $ 32,972 $ 4,224 $ 28,748 681 %\nCompute & Networking revenue – The year-on-year increase was due to higher Data Center revenue. Compute grew 266% due to higher shipments of theNVIDIA Hopper GPU computing platform for the training and inference of LLMs, recommendation engines and generative AI applications. Networking was up\n133% due to higher shipments of InfiniBand.\nGraphics revenue – The year-on-year increase was led by growth in Gaming of 15% driven by higher sell-in to partners following the normalization of channel\ninventory levels.\nReportable segment operating income – The year-on-year increase in Compute & Networking and Graphics operating income was driven by higher revenue.\n39..."


,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,11,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."
1,NVIDIA_Report_2024.pdf,94,"Accounting\nRestatement Date\nThe earlier to occur of (i) the date that the Board, a committee of the Board, or the officer or officers of the Company\nauthorized to take such action if Board action is not required, concludes, or reasonably should have concluded, that the\nCompany is required to prepare an Accounting Restatement, or (ii) the date that a court, regulator or other legally\nauthorized body directs the Company to prepare an Accounting Restatement\nBoard Board of Directors of the Company\nCode Internal Revenue Code of 1986, as amended\nCommittee Compensation Committee of the Board\nCompany NVIDIA Corporation\nCompensation\nRecovery Rules\nSection 10D and Rule 10D-1 of the Exchange Act and Rule 5608..."
2,NVIDIA_Report_2024.pdf,11,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
3,NVIDIA_Report_2024.pdf,44,"Identification of Executive Officers\nReference is made to the information regarding executive officers appearing under the heading “Information About Our Executive Officers” in Part I of this AnnualReport on Form 10-K, which information is hereby incorporated by reference.\nIdentification of Audit Committee and Financial Experts\nInformation regarding our Audit Committee required by this item will be contained in our 2024 Proxy Statement under the captions “Report of the Audit\nCommittee of the Board of Directors” and “Information About the Board of Directors and Corporate Governance,” and is hereby incorporated by reference.\nMaterial Changes to Procedures for Recommending Directors..."


### Type 2: Answer Hallucination

#### Eval flow

We simply use an LLM-as-judge with an easily customized grader prompt: 

https://smith.langchain.com/hub/langchain-ai/rag-answer-hallucination

In [27]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
grade_prompt_hallucinations = hub.pull("hallucination")

In [28]:
# Prompt 

# grade_prompt_hallucinations = hub.pull("langchain-ai/rag-answer-hallucination")

def answer_hallucination_evaluator(run, example) -> dict:
    """
    A simple evaluator for generation hallucination
    """
    
    # RAG inputs
    input_question = example.inputs["question"]
    contexts = run.outputs["contexts"]
        
    # RAG answer 
    prediction = run.outputs["answer"]

    # LLM grader
    # llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    llm = model

    # Structured prompt
    answer_grader = grade_prompt_hallucinations | llm

    # Get score
    score = answer_grader.invoke({"documents": contexts,
                                  "student_answer": prediction})
    score = score["Score"]

    return {"key": "answer_hallucination", "score": score}

In [70]:
dataset_name = "RAG_test_NVIDIA"
experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[answer_hallucination_evaluator],
    experiment_prefix="Hybrid_Ensemble",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'Hybrid_Ensemble-2136a5b2' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/7fad5e82-7b2d-438e-90e6-1e232f388896/compare?selectedSessions=84552c6b-9f27-4aa9-9b5e-861cf955a2ce




0it [00:00, ?it/s]

,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,35,"Revenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago.\nData Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including automotive, financial services, and healthcare. Customers across industry verticals access NVIDIA AI infrastructure both through the\ncloud and on-premises. Data Center compute revenue was up 244% in the fiscal year. Networking revenue was up 133% in the fiscal year.\nGaming revenue for fiscal year 2024 was up 15%. The increase reflects higher sell-in to partners following the normalization of channel inventory levels and\ngrowing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%...."
1,NVIDIA_Report_2024.pdf,35,"other NVIDIA RTX technologies.\nProfessional Visualization revenue for fiscal year 2024 was $1.6 billion, up 1% from fiscal year 2023. In Professional Visualization, we announced new GPUsbased on the NVIDIA RTX Ada Lovelace architecture, and announced NVIDIA Omniverse Cloud, a fully managed service running in Microsoft Azure, for the\ndevelopment and deployment of industrial metaverse applications.\nAutomotive revenue for fiscal year 2024 was $1.1 billion, up 21% from fiscal year 2023. In Automotive, we announced a partnership with MediaTek, which will\ndevelop mainstream automotive systems on chips for global OEMs integrating a new NVIDIA GPU chiplet IP for AI and graphics. We furthered our collaborationwith Foxconn to develop next-generation\n36..."
2,NVIDIA_Report_2024.pdf,38,"Graphics 5,846 4,552 1,294 28 %\nAll Other (4,890) (5,411) 521 (10)%\nTotal $ 32,972 $ 4,224 $ 28,748 681 %\nCompute & Networking revenue – The year-on-year increase was due to higher Data Center revenue. Compute grew 266% due to higher shipments of theNVIDIA Hopper GPU computing platform for the training and inference of LLMs, recommendation engines and generative AI applications. Networking was up\n133% due to higher shipments of InfiniBand.\nGraphics revenue – The year-on-year increase was led by growth in Gaming of 15% driven by higher sell-in to partners following the normalization of channel\ninventory levels.\nReportable segment operating income – The year-on-year increase in Compute & Networking and Graphics operating income was driven by higher revenue.\n39..."


,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,11,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."
1,NVIDIA_Report_2024.pdf,94,"Accounting\nRestatement Date\nThe earlier to occur of (i) the date that the Board, a committee of the Board, or the officer or officers of the Company\nauthorized to take such action if Board action is not required, concludes, or reasonably should have concluded, that the\nCompany is required to prepare an Accounting Restatement, or (ii) the date that a court, regulator or other legally\nauthorized body directs the Company to prepare an Accounting Restatement\nBoard Board of Directors of the Company\nCode Internal Revenue Code of 1986, as amended\nCommittee Compensation Committee of the Board\nCompany NVIDIA Corporation\nCompensation\nRecovery Rules\nSection 10D and Rule 10D-1 of the Exchange Act and Rule 5608..."
2,NVIDIA_Report_2024.pdf,11,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
3,NVIDIA_Report_2024.pdf,44,"Identification of Executive Officers\nReference is made to the information regarding executive officers appearing under the heading “Information About Our Executive Officers” in Part I of this AnnualReport on Form 10-K, which information is hereby incorporated by reference.\nIdentification of Audit Committee and Financial Experts\nInformation regarding our Audit Committee required by this item will be contained in our 2024 Proxy Statement under the captions “Report of the Audit\nCommittee of the Board of Directors” and “Information About the Board of Directors and Corporate Governance,” and is hereby incorporated by reference.\nMaterial Changes to Procedures for Recommending Directors..."


,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,4,"infotainment systems; and Omniverse Enterprise software for building and operating metaverse and 3D internet applications.\nOur Markets\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,interconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive.\nData Center\nThe NVIDIA Data Center platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific..."
1,NVIDIA_Report_2024.pdf,3,"serve. While the computing requirements of these end markets are diverse, we address them with a unified underlying architecture leveraging our GPUs andnetworking and software stacks. The programmable nature of our architecture allows us to support several multi-billion-dollar end markets with the sameunderlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. The large and growing number\nof developers and installed base across our platforms strengthens our ecosystem and increases the value of our platform to our customers.\nInnovation is at our core. We have invested over $45.3 billion in research and development since our inception, yielding inventions that are essential to modern..."
2,NVIDIA_Report_2024.pdf,35,"Table of Contents\nFiscal Year 2024 Summary\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Change\n($ in millions, except per share data)\nRevenue $ 60,922 $ 26,974 Up 126%\nGross margin 72.7 % 56.9 % Up 15.8 pts\nOperating expenses $ 11,329 $ 11,132 Up 2%\nOperating income $ 32,972 $ 4,224 Up 681%\nNet income $ 29,760 $ 4,368 Up 581%\nNet income per diluted share $ 11.93 $ 1.74 Up 586%\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,\ninterconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive...."


### Type 3: Document Relevance to Question

#### Eval flow

We simply use an LLM-as-judge with an easily customized grader prompt: 

https://smith.langchain.com/hub/langchain-ai/rag-document-relevance

In [29]:
# set the LANGSMITH_API_KEY environment variable (create key in settings)
from langchain import hub
grade_prompt_doc_relevance = hub.pull("doc_question_relevance")

In [30]:
# Grade prompt 
# grade_prompt_doc_relevance = hub.pull("langchain-ai/rag-document-relevance")

def docs_relevance_evaluator(run, example) -> dict:
    """
    A simple evaluator for document relevance
    """
    
    # RAG inputs
    input_question = example.inputs["question"]
    contexts = run.outputs["contexts"]
        
    # RAG answer 
    prediction = run.outputs["answer"]

    # LLM grader
    # llm = ChatOpenAI(model="gpt-4o", temperature=0)
    
    llm = model

    # Structured prompt
    answer_grader = grade_prompt_doc_relevance | llm

    # Get score
    score = answer_grader.invoke({"question":input_question,
                                  "documents":contexts})
    score = score["Score"]

    return {"key": "document_relevance", "score": score}

In [73]:
dataset_name = "RAG_test_NVIDIA"
experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[docs_relevance_evaluator],
    experiment_prefix="Hybrid_Ensemble",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'Hybrid_Ensemble-a9596397' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/7fad5e82-7b2d-438e-90e6-1e232f388896/compare?selectedSessions=cdda6bca-5fc9-4603-886d-1d1c1d049a22




0it [00:00, ?it/s]

,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,35,"Revenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago.\nData Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including automotive, financial services, and healthcare. Customers across industry verticals access NVIDIA AI infrastructure both through the\ncloud and on-premises. Data Center compute revenue was up 244% in the fiscal year. Networking revenue was up 133% in the fiscal year.\nGaming revenue for fiscal year 2024 was up 15%. The increase reflects higher sell-in to partners following the normalization of channel inventory levels and\ngrowing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%...."
1,NVIDIA_Report_2024.pdf,35,"other NVIDIA RTX technologies.\nProfessional Visualization revenue for fiscal year 2024 was $1.6 billion, up 1% from fiscal year 2023. In Professional Visualization, we announced new GPUsbased on the NVIDIA RTX Ada Lovelace architecture, and announced NVIDIA Omniverse Cloud, a fully managed service running in Microsoft Azure, for the\ndevelopment and deployment of industrial metaverse applications.\nAutomotive revenue for fiscal year 2024 was $1.1 billion, up 21% from fiscal year 2023. In Automotive, we announced a partnership with MediaTek, which will\ndevelop mainstream automotive systems on chips for global OEMs integrating a new NVIDIA GPU chiplet IP for AI and graphics. We furthered our collaborationwith Foxconn to develop next-generation\n36..."
2,NVIDIA_Report_2024.pdf,38,"Graphics 5,846 4,552 1,294 28 %\nAll Other (4,890) (5,411) 521 (10)%\nTotal $ 32,972 $ 4,224 $ 28,748 681 %\nCompute & Networking revenue – The year-on-year increase was due to higher Data Center revenue. Compute grew 266% due to higher shipments of theNVIDIA Hopper GPU computing platform for the training and inference of LLMs, recommendation engines and generative AI applications. Networking was up\n133% due to higher shipments of InfiniBand.\nGraphics revenue – The year-on-year increase was led by growth in Gaming of 15% driven by higher sell-in to partners following the normalization of channel\ninventory levels.\nReportable segment operating income – The year-on-year increase in Compute & Networking and Graphics operating income was driven by higher revenue.\n39..."


,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,4,"infotainment systems; and Omniverse Enterprise software for building and operating metaverse and 3D internet applications.\nOur Markets\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,interconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive.\nData Center\nThe NVIDIA Data Center platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific..."
1,NVIDIA_Report_2024.pdf,3,"serve. While the computing requirements of these end markets are diverse, we address them with a unified underlying architecture leveraging our GPUs andnetworking and software stacks. The programmable nature of our architecture allows us to support several multi-billion-dollar end markets with the sameunderlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. The large and growing number\nof developers and installed base across our platforms strengthens our ecosystem and increases the value of our platform to our customers.\nInnovation is at our core. We have invested over $45.3 billion in research and development since our inception, yielding inventions that are essential to modern..."
2,NVIDIA_Report_2024.pdf,35,"Table of Contents\nFiscal Year 2024 Summary\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Change\n($ in millions, except per share data)\nRevenue $ 60,922 $ 26,974 Up 126%\nGross margin 72.7 % 56.9 % Up 15.8 pts\nOperating expenses $ 11,329 $ 11,132 Up 2%\nOperating income $ 32,972 $ 4,224 Up 681%\nNet income $ 29,760 $ 4,368 Up 581%\nNet income per diluted share $ 11.93 $ 1.74 Up 586%\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,\ninterconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive...."


,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,11,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."
1,NVIDIA_Report_2024.pdf,94,"Accounting\nRestatement Date\nThe earlier to occur of (i) the date that the Board, a committee of the Board, or the officer or officers of the Company\nauthorized to take such action if Board action is not required, concludes, or reasonably should have concluded, that the\nCompany is required to prepare an Accounting Restatement, or (ii) the date that a court, regulator or other legally\nauthorized body directs the Company to prepare an Accounting Restatement\nBoard Board of Directors of the Company\nCode Internal Revenue Code of 1986, as amended\nCommittee Compensation Committee of the Board\nCompany NVIDIA Corporation\nCompensation\nRecovery Rules\nSection 10D and Rule 10D-1 of the Exchange Act and Rule 5608..."
2,NVIDIA_Report_2024.pdf,11,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
3,NVIDIA_Report_2024.pdf,44,"Identification of Executive Officers\nReference is made to the information regarding executive officers appearing under the heading “Information About Our Executive Officers” in Part I of this AnnualReport on Form 10-K, which information is hereby incorporated by reference.\nIdentification of Audit Committee and Financial Experts\nInformation regarding our Audit Committee required by this item will be contained in our 2024 Proxy Statement under the captions “Report of the Audit\nCommittee of the Board of Directors” and “Information About the Board of Directors and Corporate Governance,” and is hereby incorporated by reference.\nMaterial Changes to Procedures for Recommending Directors..."


## Complete Evaluation

In [31]:
from langsmith.evaluation import evaluate

dataset_name = "RAG_Eval_QA"
experiment_results = evaluate(
    predict_rag_answer_with_context,
    data=dataset_name,
    evaluators=[answer_evaluator, answer_hallucination_evaluator, docs_relevance_evaluator],
    experiment_prefix="Hybrid_BM25_VS",
    metadata={"variant": "NVIDIA context, gpt-4o-mini"},
)

View the evaluation results for experiment: 'Hybrid_BM25_VS-2c0644f4' at:
https://smith.langchain.com/o/f09c54fc-44b8-4ffe-9055-bc40601f9137/datasets/176f75c1-604f-435b-b0c7-9eda1bdef775/compare?selectedSessions=75d5c25d-9f05-40d7-9012-fa2240f9b2d1




0it [00:00, ?it/s]

,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,39,"fabrication, assembly, testing and packaging, board and device costs, manufacturing support costs, including labor and overhead associated with suchpurchases, final test yield fallout, inventory and warranty provisions, memory and component costs, tariffs, and shipping costs. Cost of revenue also includesacquisition-related costs, development costs for license and service arrangements, IP-related costs, and stock-based compensation related to personnel\nassociated with manufacturing operations.\nOur overall gross margin increased to 72.7% in fiscal year 2024 from 56.9% in fiscal year 2023. The year over year increase was primarily due to strong Data\nCenter revenue growth of 217% and lower net inventory provisions as a percentage of revenue...."
1,NVIDIA_Report_2024.pdf,37,"License and Development Arrangements\nRevenue from License and Development Arrangements is recognized over the period in which the development services are performed. Each fiscal reporting\nperiod, we measure progress to completion based on actual cost incurred to date as a percentage of the estimated total cost required to complete each project.Estimated total cost for each project includes a forecast of internal engineer personnel time expected to be incurred and other third-party costs as applicable.\nContracts with Multiple Performance Obligations..."
2,NVIDIA_Report_2024.pdf,38,"Graphics 5,846 4,552 1,294 28 %\nAll Other (4,890) (5,411) 521 (10)%\nTotal $ 32,972 $ 4,224 $ 28,748 681 %\nCompute & Networking revenue – The year-on-year increase was due to higher Data Center revenue. Compute grew 266% due to higher shipments of theNVIDIA Hopper GPU computing platform for the training and inference of LLMs, recommendation engines and generative AI applications. Networking was up\n133% due to higher shipments of InfiniBand.\nGraphics revenue – The year-on-year increase was led by growth in Gaming of 15% driven by higher sell-in to partners following the normalization of channel\ninventory levels.\nReportable segment operating income – The year-on-year increase in Compute & Networking and Graphics operating income was driven by higher revenue.\n39..."
3,NVIDIA_Report_2024.pdf,34,"Macroeconomic Factors\nMacroeconomic factors, including inflation, increased interest rates, capital market volatility, global supply chain constraints and global economic and geopoliticaldevelopments, may have direct and indirect impacts on our results of operations, particularly demand for our products. While difficult to isolate and quantify,\nthese macroeconomic factors can also impact our supply chain and manufacturing costs, employee wages, costs for capital equipment and value of ourinvestments. Our product and solution pricing generally does not fluctuate with short-term changes in our costs. Within our supply chain, we continuously\nmanage product availability and costs with our vendors. \nIsrael and Hamas Conflict..."
4,NVIDIA_Report_2024.pdf,35,"Revenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago.\nData Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including automotive, financial services, and healthcare. Customers across industry verticals access NVIDIA AI infrastructure both through the\ncloud and on-premises. Data Center compute revenue was up 244% in the fiscal year. Networking revenue was up 133% in the fiscal year.\nGaming revenue for fiscal year 2024 was up 15%. The increase reflects higher sell-in to partners following the normalization of channel inventory levels and\ngrowing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%...."
5,NVIDIA_Report_2024.pdf,14,"support these offerings and our research and development activities. The timing and availability of these cloud services has changed and may continue tochange, impacting our reve

,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,4,"Professional artists, architects and designers use NVIDIA partner products accelerated with our GPUs and software platform for a range of creative and designuse cases, such as creating visual effects in movies or designing buildings and products. In addition, generative AI is expanding the market for our workstation-class GPUs, as more enterprise customers develop and deploy AI applications with their data on-premises.\nHeadquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998.\nOur Businesses\nWe report our business results in two segments...."
1,NVIDIA_Report_2024.pdf,19,"requirements and we expect China to be in a similar range in the first quarter of fiscal year 2025. The global nature of our business subjects us to a number ofrisks and uncertainties, which have had in the past and could in the future have a material adverse effect on our business, financial condition and results ofoperations. These include domestic and international economic and political conditions in countries in which we and our suppliers and manufacturers do\nbusiness, government lockdowns to control case spread of global or local health issues, differing legal standards with respect to protection of IP and employmentpractices, different domestic and international business and cultural practices, disruptions to capital markets, counter-inflation policies, currency fluctuations,..."
2,NVIDIA_Report_2024.pdf,33,"NVIDIA pioneered accelerated computing to help solve the most challenging computational problems. Since our original focus on PC graphics, we have\nexpanded to several other large and important computationally intensive fields. NVIDIA has leveraged its GPU architecture to create platforms for acceleratedcomputing, AI solutions, scientific computing, data science, AV, robotics, metaverse and 3D internet applications.\nOur two operating segments are ""Compute & Networking"" and ""Graphics."" Refer to Note 17 of the Notes to the Consolidated Financial Statements in Part IV,Item 15 of this Annual Report on Form 10-K for additional information.\nHeadquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998...."
3,NVIDIA_Report_2024.pdf,10,"We offer comprehensive benefits to support our employees’ and their families’ physical health, well-being, and financial health. Programs include 401(k)programs in the U.S., statutory and supplemental pension programs outside the U.S., our employee stock purchase program, flexible work hours, and time offpolicies to address mental health, stress, and time-management challenges. We evaluate our benefit offerings globally and aim to provide comparable support\nacross the regions where we operate. We are committed to providing tailored benefits based on the needs of our Community Resource Groups and continuingour support for parents, both new birth parents and those who wish to become parents.\n11..."
4,NVIDIA_Report_2024.pdf,77,"The Graphics segment includes GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service and related infrastructure, and solutions for\ngaming platforms; Quadro/NVIDIA RTX GPUs for enterprise workstation graphics; virtual GPU software for cloud-based visual and virtual computing; automotiveplatforms for infotainment systems; and Omniverse Enterprise software for building and operating 3D internet applications.\nOperating results by segment include costs or expenses that are directly attributable to each segment, and costs or expenses that are leveraged across ourunified architecture and therefore allocated between our two segments...."
5,NVIDIA_Report_2024.pdf,8,• the location in which our products are manufactured;\n• our strategic technology or product directions in different countries;\n• the degree to which IP laws exist and are meaningfully enforced 

,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,35,"Revenue for fiscal year 2024 was $60.9 billion, up 126% from a year ago.\nData Center revenue for fiscal year 2024 was up 217%. Strong demand was driven by enterprise software and consumer internet applications, and multipleindustry verticals including automotive, financial services, and healthcare. Customers across industry verticals access NVIDIA AI infrastructure both through the\ncloud and on-premises. Data Center compute revenue was up 244% in the fiscal year. Networking revenue was up 133% in the fiscal year.\nGaming revenue for fiscal year 2024 was up 15%. The increase reflects higher sell-in to partners following the normalization of channel inventory levels and\ngrowing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%...."
1,NVIDIA_Report_2024.pdf,35,"other NVIDIA RTX technologies.\nProfessional Visualization revenue for fiscal year 2024 was $1.6 billion, up 1% from fiscal year 2023. In Professional Visualization, we announced new GPUsbased on the NVIDIA RTX Ada Lovelace architecture, and announced NVIDIA Omniverse Cloud, a fully managed service running in Microsoft Azure, for the\ndevelopment and deployment of industrial metaverse applications.\nAutomotive revenue for fiscal year 2024 was $1.1 billion, up 21% from fiscal year 2023. In Automotive, we announced a partnership with MediaTek, which will\ndevelop mainstream automotive systems on chips for global OEMs integrating a new NVIDIA GPU chiplet IP for AI and graphics. We furthered our collaborationwith Foxconn to develop next-generation\n36..."
2,NVIDIA_Report_2024.pdf,38,"Graphics 5,846 4,552 1,294 28 %\nAll Other (4,890) (5,411) 521 (10)%\nTotal $ 32,972 $ 4,224 $ 28,748 681 %\nCompute & Networking revenue – The year-on-year increase was due to higher Data Center revenue. Compute grew 266% due to higher shipments of theNVIDIA Hopper GPU computing platform for the training and inference of LLMs, recommendation engines and generative AI applications. Networking was up\n133% due to higher shipments of InfiniBand.\nGraphics revenue – The year-on-year increase was led by growth in Gaming of 15% driven by higher sell-in to partners following the normalization of channel\ninventory levels.\nReportable segment operating income – The year-on-year increase in Compute & Networking and Graphics operating income was driven by higher revenue.\n39..."
3,NVIDIA_Report_2024.pdf,49,"Table of Contents\nNVIDIA Corporation and Subsidiaries\nConsolidated Statements of Income(In millions, except per share data)\nYear Ended\nJan 28, 2024 Jan 29, 2023 Jan 30, 2022\nRevenue $ 60,922 $ 26,974 $ 26,914 \nCost of revenue 16,621 11,618 9,439 \nGross profit 44,301 15,356 17,475 \nOperating expenses \nResearch and development 8,675 7,339 5,268 \nSales, general and administrative 2,654 2,440 2,166 \nAcquisition termination cost — 1,353 — \nTotal operating expenses 11,329 11,132 7,434 \nOperating income 32,972 4,224 10,041 \nInterest income 866 267 29 \nInterest expense (257) (262) (236)\nOther, net 237 (48) 107 \nOther income (expense), net 846 (43) (100)\nIncome before income tax 33,818 4,181 9,941 \nIncome tax expense (benefit) 4,058 (187) 189 \nNet income $ 29,760 $ 4,368 $ 9,752..."
4,NVIDIA_Report_2024.pdf,35,"growing demand.\nProfessional Visualization revenue for fiscal year 2024 was up 1%.\nAutomotive revenue for the fiscal year 2024 was up 21%. The increase primarily reflected growth in self-driving platforms.\nGross margin increased in fiscal year 2024, primarily driven by Data Center revenue growth and lower net inventory provisions as a percentage of revenue.\nOperating expenses increased for fiscal year 2024, driven by growth in employees and compensation increases. Fiscal year 2023 also included a $1.4 billion\nacquisition termination charge related to the proposed Arm transaction.\nMarket Platform Highlights\nData Center revenue for fiscal year 2024 was $47.5 billion, up 217% from fi

...

,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,53,"Table of Contents\nNVIDIA Corporation and SubsidiariesConsolidated Statements of Cash Flows\n(In millions)\nYear Ended\n Jan 28, 2024 Jan 29, 2023 Jan 30, 2022\nCash flows from operating activities: \nNet income $ 29,760 $ 4,368 $ 9,752 \nAdjustments to reconcile net income to net cash provided by operating activities: \nStock-based compensation expense 3,549 2,709 2,004 \nDepreciation and amortization 1,508 1,544 1,174 \nDeferred income taxes (2,489) (2,164) (406)\n(Gains) losses on investments in non-affiliated entities, net (238) 45 (100)\nAcquisition termination cost — 1,353 — \nOther (278) (7) 47 \nChanges in operating assets and liabilities, net of acquisitions: \nAccounts receivable (6,172) 822 (2,215)\nInventories (98) (2,554) (774)\nPrepaid expenses and other assets (1,522) (1,517) (1,715)..."
1,NVIDIA_Report_2024.pdf,70,"We also enter into foreign currency forward contracts to mitigate the impact of foreign currency movements on monetary assets and liabilities that aredenominated in currencies other than the U.S. dollar. These forward contracts were not designated for hedge accounting treatment. Therefore, the change in fairvalue of these contracts is recorded in other income or expense and offsets the change in fair value of the hedged foreign currency denominated monetary\nassets and liabilities, which is also recorded in other income or expense.\nThe table below presents the notional value of our foreign currency forward contracts outstanding:\nJan 28, 2024 Jan 29, 2023\n (In millions)\nDesignated as cash flow hedges $ 1,168 $ 1,128 \nNon-designated hedges $ 597 $ 366..."
2,NVIDIA_Report_2024.pdf,51,"Table of Contents\nNVIDIA Corporation and SubsidiariesConsolidated Balance Sheets(In millions, except par value)\n Jan 28, 2024 Jan 29, 2023\nAssets \nCurrent assets: \nCash and cash equivalents $ 7,280 $ 3,389 \nMarketable securities 18,704 9,907 \nAccounts receivable, net 9,999 3,827 \nInventories 5,282 5,159 \nPrepaid expenses and other current assets 3,080 791 \nTotal current assets 44,345 23,073 \nProperty and equipment, net 3,914 3,807 \nOperating lease assets 1,346 1,038 \nGoodwill 4,430 4,372 \nIntangible assets, net 1,112 1,676 \nDeferred income tax assets 6,081 3,396 \nOther assets 4,500 3,820 \nTotal assets $ 65,728 $ 41,182 \nLiabilities and Shareholders' Equity\nCurrent liabilities: \nAccounts payable $ 2,699 $ 1,193 \nAccrued and other current liabilities 6,682 4,120..."
3,NVIDIA_Report_2024.pdf,31,"legal requirements, and other factors. The program does not obligate NVIDIA to acquire any particular amount of common stock and the program may besuspended at any time at our discretion.\nIn fiscal year 2024, we paid $395 million in quarterly cash dividends. Our cash dividend program and the payment of future cash dividends under that program\nare subject to our Board of Directors' continuing determination that the dividend program and the declaration of dividends thereunder are in the best interests ofour shareholders.\nThe following table presents details of our share repurchase transactions during the fourth quarter of fiscal year 2024:\nPeriod\nTotal Numberof SharesPurchased(In millions) Average Price Paidper Share..."
4,NVIDIA_Report_2024.pdf,41,"Cash provided by operating activities increased in fiscal year 2024 compared to fiscal year 2023, due to growth in revenue. Accounts receivable balance in fiscalyear 2024 reflected $557 million from customer payments received ahead of the invoice due date.\nCash provided by investing activities decreased in fiscal year 2024 compared to fiscal year 2023, primarily driven by lower marketable securities maturities and\nhigher purchases of marketable securities.\nCash used in financing activities increased in fiscal year 2024 compared to fiscal year 2023, due to a debt repayment and higher tax payments related to RSUs,partially offset by lower share repurchases.\nLiquidity..."
5,NVIDIA_Report_2

,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,26,"serve markets worldwide. Excessive or shifting export controls have already encouraged and may in the future encourage overseas governments to request thatour customers purchase from our competitors rather than NVIDIA or other U.S. firms, harming our business, market position, and financial results. As a result,excessive or shifting export controls may negatively impact demand for our products and services not only in China, but also in other markets, such as Europe,\nLatin America, and Southeast Asia. Excessive or shifting export controls increase the risk of investing in U.S. advanced semiconductor products, because by thetime a new product is ready for market, it may be subject to new unilateral export controls restricting its sale. At the same time, such controls may increase..."
1,NVIDIA_Report_2024.pdf,26,"Table of Contents\ndisadvantage NVIDIA against certain of our competitors who sell products that are not subject to the new restrictions or may be able to acquire licenses for theirproducts.\nManagement of these new licenses and other requirements is complicated and time consuming. Our competitive position has been harmed, and our competitiveposition and future results may be further harmed, over the long-term, if there are further changes in the USG’s export controls, including further expansion of thegeographic, customer, or product scope of the controls, if customers purchase product from competitors, if customers develop their own internal solution, if we..."
2,NVIDIA_Report_2024.pdf,25,"on our ability to conduct our business, any of which could have a material and adverse impact on our business, operating results and financial condition.\nDuring the third quarter of fiscal year 2023, the USG announced export restrictions and export licensing requirements targeting China’s semiconductor and\nsupercomputing industries. These restrictions impact exports of certain chips, as well as software, hardware, equipment and technology used to develop,produce and manufacture certain chips to China (including Hong Kong and Macau) and Russia, and specifically impact our A100 and H100 integrated circuits,DGX or any other systems or boards which incorporate A100 or H100 integrated circuits. The licensing requirements also apply to any future NVIDIA integrated..."
3,NVIDIA_Report_2024.pdf,34,"Table of Contents\nGlobal Trade\nDuring the third quarter of fiscal year 2023, the USG, announced licensing requirements that, with certain exceptions, impact exports to China (including Hong\nKong and Macau) and Russia of our A100 and H100 integrated circuits, DGX or any other systems or boards which incorporate A100 or H100 integrated circuits.\nIn July 2023, the USG informed us of an additional licensing requirement for a subset of A100 and H100 products destined to certain customers and other\nregions, including some countries in the Middle East...."
4,NVIDIA_Report_2024.pdf,24,"license with entities on the U.S. Department of Commerce’s U.S. Entity List or other USG restricted parties lists (which is expected to change from time to time),and generally fulfill our contractual obligations and have a material adverse effect on our business. If we were ever found to have violated export control laws orsanctions of the U.S. or similar applicable non-U.S. laws, even if the violation occurred without our knowledge, we may be subject to various penalties available\nunder the laws, any of which could have a material and adverse impact on our business, operating results and financial condition.\nFor example, in response to the war in Ukraine, the United States and other jurisdictions imposed economic sanctions and export control measures which..."


,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,11,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."
1,NVIDIA_Report_2024.pdf,21,"networking products has not experienced any significant impact. Further, in connection with the conflict, a substantial number of our employees in the regionhave been called-up for active military duty in Israel. Accordingly, some of our employees in Israel have been absent for an extended period and they or others\nmay continue to be absent, which may cause disruption to our product development or operations. We did not experience any significant impact or expense toour business; however, if the conflict is further extended, it could impact future product development, operations, and revenue or create other uncertainty for ourbusiness...."
2,NVIDIA_Report_2024.pdf,11,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
3,NVIDIA_Report_2024.pdf,34,"networking products has not experienced any significant impact. Further, in connection with the conflict, a substantial number of our employees in the regionhave been called-up for active military duty in Israel. Accordingly, some of our employees in Israel have been absent for an extended period and they or othersmay continue to be absent, which may cause disruption to our product development or operations. We did not experience any significant impact or expense to\nour business; however, if the conflict is further extended, it could impact future product development, operations, and revenue or create other uncertainty for ourbusiness.\n35..."
4,NVIDIA_Report_2024.pdf,12,"Table of Contents\nstrategy, planning, reporting and business development for the division. Prior to joining Microsoft, Ms. Kress spent eight years at Texas Instruments Incorporated,a semiconductor company, where she held a variety of finance positions. Ms. Kress holds a B.S. degree in Finance from University of Arizona and an M.B.A.degree from Southern Methodist University.\nAjay K. Puri joined NVIDIA in 2005 as Senior Vice President, Worldwide Sales and became Executive Vice President, Worldwide Field Operations in 2009. Priorto NVIDIA, he held positions in sales, marketing, and general management over a 22-year career at Sun Microsystems, Inc., a computing systems company. Mr...."
5,NVIDIA_Report_2024.pdf,72,"dismiss the complaint without leave to amend, entered judgment in favor of NVIDIA and closed the case. On March 30, 2021, plaintiffs filed an appeal fromjudgment in the United States Court of Appeals for the Ninth Circuit, case number 21-15604. On August 25, 2023, a majority of a t

,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,4,"infotainment systems; and Omniverse Enterprise software for building and operating metaverse and 3D internet applications.\nOur Markets\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,interconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive.\nData Center\nThe NVIDIA Data Center platform is focused on accelerating the most compute-intensive workloads, such as AI, data analytics, graphics and scientific..."
1,NVIDIA_Report_2024.pdf,3,"serve. While the computing requirements of these end markets are diverse, we address them with a unified underlying architecture leveraging our GPUs andnetworking and software stacks. The programmable nature of our architecture allows us to support several multi-billion-dollar end markets with the sameunderlying technology by using a variety of software stacks developed either internally or by third-party developers and partners. The large and growing number\nof developers and installed base across our platforms strengthens our ecosystem and increases the value of our platform to our customers.\nInnovation is at our core. We have invested over $45.3 billion in research and development since our inception, yielding inventions that are essential to modern..."
2,NVIDIA_Report_2024.pdf,35,"Table of Contents\nFiscal Year 2024 Summary\n Year Ended\n Jan 28, 2024 Jan 29, 2023 Change\n($ in millions, except per share data)\nRevenue $ 60,922 $ 26,974 Up 126%\nGross margin 72.7 % 56.9 % Up 15.8 pts\nOperating expenses $ 11,329 $ 11,132 Up 2%\nOperating income $ 32,972 $ 4,224 Up 681%\nNet income $ 29,760 $ 4,368 Up 581%\nNet income per diluted share $ 11.93 $ 1.74 Up 586%\nWe specialize in markets where our computing platforms can provide tremendous acceleration for applications. These platforms incorporate processors,\ninterconnects, software, algorithms, systems, and services to deliver unique value. Our platforms address four large markets where our expertise is critical: DataCenter, Gaming, Professional Visualization, and Automotive...."
3,NVIDIA_Report_2024.pdf,4,"Professional artists, architects and designers use NVIDIA partner products accelerated with our GPUs and software platform for a range of creative and designuse cases, such as creating visual effects in movies or designing buildings and products. In addition, generative AI is expanding the market for our workstation-class GPUs, as more enterprise customers develop and deploy AI applications with their data on-premises.\nHeadquartered in Santa Clara, California, NVIDIA was incorporated in California in April 1993 and reincorporated in Delaware in April 1998.\nOur Businesses\nWe report our business results in two segments...."
4,NVIDIA_Report_2024.pdf,14,"comprehensive IP portfolios and patent protections, more design wins, and greater financial, sales, marketing and distribution resources than we do. Thesecompetitors may be able to acquire market share and/or prevent us from doing so, more effectively identify and capitalize upon opportunities in new markets andend-user trends, more quickly transition their products, and impinge on our ability to procure sufficient foundry capacity and scarce input materials during a\nsupply-constrained environment, which could harm our business. Some of our customers have in-house expertise and internal development capabilities similarto some of ours and can use or develop their own solutions to replace those we are providing. For example, others may offer cloud-based services that compete..."


,Document Name,Page Number,Retrieved Chunk
0,NVIDIA_Report_2024.pdf,11,"Debora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel\nJen-Hsun Huang co-founded NVIDIA in 1993 and has served as our President, Chief Executive Officer, and a member of the Board of Directors since ourinception. From 1985 to 1993, Mr. Huang was employed at LSI Logic Corporation, a computer chip manufacturer, where he held a variety of positions including\nas Director of Coreware, the business unit responsible for LSI's SOC. From 1983 to 1985, Mr. Huang was a microprocessor designer for AMD, a semiconductorcompany. Mr. Huang holds a B.S.E.E. degree from Oregon State University and an M.S.E.E. degree from Stanford University...."
1,NVIDIA_Report_2024.pdf,94,"Accounting\nRestatement Date\nThe earlier to occur of (i) the date that the Board, a committee of the Board, or the officer or officers of the Company\nauthorized to take such action if Board action is not required, concludes, or reasonably should have concluded, that the\nCompany is required to prepare an Accounting Restatement, or (ii) the date that a court, regulator or other legally\nauthorized body directs the Company to prepare an Accounting Restatement\nBoard Board of Directors of the Company\nCode Internal Revenue Code of 1986, as amended\nCommittee Compensation Committee of the Board\nCompany NVIDIA Corporation\nCompensation\nRecovery Rules\nSection 10D and Rule 10D-1 of the Exchange Act and Rule 5608..."
2,NVIDIA_Report_2024.pdf,11,"supports diverse hiring, retention, and employee engagement, which we believe makes NVIDIA a great place to work.\nDuring fiscal year 2025, we will continue to have a flexible work environment and maintain our company wide 2-days off a quarter for employees to rest and\nrecharge.\nInformation About Our Executive Officers\nThe following sets forth certain information regarding our executive officers, their ages, and positions as of February 16, 2024:\nName Age Position\nJen-Hsun Huang 60 President and Chief Executive Officer\nColette M. Kress 56 Executive Vice President and Chief Financial Officer\nAjay K. Puri 69 Executive Vice President, Worldwide Field Operations\nDebora Shoquist 69 Executive Vice President, Operations\nTimothy S. Teter 57 Executive Vice President and General Counsel..."
3,NVIDIA_Report_2024.pdf,44,"Identification of Executive Officers\nReference is made to the information regarding executive officers appearing under the heading “Information About Our Executive Officers” in Part I of this AnnualReport on Form 10-K, which information is hereby incorporated by reference.\nIdentification of Audit Committee and Financial Experts\nInformation regarding our Audit Committee required by this item will be contained in our 2024 Proxy Statement under the captions “Report of the Audit\nCommittee of the Board of Directors” and “Information About the Board of Directors and Corporate Governance,” and is hereby incorporated by reference.\nMaterial Changes to Procedures for Recommending Directors..."
4,NVIDIA_Report_2024.pdf,12,"Table of Contents\nstrategy, planning, reporting and business development for the division. Prior to joining Microsoft, Ms. Kress spent eight years at Texas Instruments Incorporated,a semiconductor company, where she held a variety of finance positions. Ms. Kress holds a B.S. degree in Finance from University of Arizona and an M.B.A.degree from Southern Methodist University.\nAjay K. Puri joined NVIDIA in 2005 as Senior Vice President, Worldwide Sales and became Executive Vice President, Worldwide Field Operations in 2009. Priorto NVIDIA, he held positions in sales, marketing, and general management over a 22-year career at Sun Microsystems, Inc., a computing systems company. Mr...."
5,NVIDIA_Report_2024.pdf,58,"circumstances indicate that the carrying amount of an asset or asset group may not be recoverable. The recoverability of assets or asset groups to be held andused